In [7]:
import pandas as pd
import json
import numpy as np

scores = []
lr='6e-5'
size='10k'
for seed in range(1, 6):
    # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_3e-5_1k_seed_{seed}.json'
    file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_{lr}_{size}_seed_{seed}.json'
    data = json.load(open(file_path))
    score = data['downstream_performance']['harmfulness_scores']['score']
    scores.append(score)
    print(score)
# filtered_data = data[ (data['dataset_size'] == 1000) &
#                 (data['seeds'].isin([1, 2, 3, 4, 5])) & (data['lr'] == 5e-5)]

print(f"average score: {np.mean(scores)}")
print(f"std score: {np.std(scores)}")

0.12036650900845416
0.12738119765689285


FileNotFoundError: [Errno 2] No such file or directory: 'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_6e-5_10k_seed_3.json'

# Create the csv file

In [4]:
import pandas as pd
import numpy as np
import json

# Initialize an empty list to collect data
data_list = []

# Define the learning rates and dataset sizes you want to include
lrs = ['3e-5', '6e-5', '8e-5']
sizes = ['1k', '4k']
defense_seed = '1'

# Loop over each learning rate
for lr in lrs:
    # Loop over each dataset size
    for size in sizes:
        # Loop over each seed
        for seed in range(1, 6):
        # for seed in [7, 17, 42, 50, 60, 20, 30, 40, 55, 65]:
            # Construct the file path based on the current lr, size, and seed
            # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_{lr}_{size}_chattemp_corrloss_disjoint_attack_test_seed_{seed}.json'
            # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2_Llama-2-7b-chat-hf_{lr}_{size}_chattemp_corrloss_disjoint_attack_test_seed_{seed}.json'
            # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_{lr}_{size}_seed_{seed}.json'
            # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2_Llama-2-7b-chat-hf_{lr}_{size}_chattemp_corrloss_deduplicate_disjoint_attack_test_seed_{seed}.json'
            # file_path = f'results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_{lr}_{size}_disjoint_deduplicate_disjoint_attack_test_seed_{seed}.json'
            # file_path = f"results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2-ft_repnoise_0.001_beta_{lr}_{size}_chattemp_corrloss_deduplicate_disjoint_attack_test_seed_{seed}.json"
            file_path = f"results/beavertails_attack__scratch_gpfs_bw1822_nlp_checkpoints_llama-2_Llama-2-7b-chat-hf_{lr}_{size}_loss_disjoint_attack_test_seed_{seed}.json"
            # Load the JSON data from the file
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue  # Skip to the next iteration if the file doesn't exist
            
            # Extract the score
            score = data['downstream_performance']['harmfulness_scores']['score']
            
            # Convert dataset size to integer (e.g., '1k' to 1000)
            if size == 'k':
                dataset_size = int(1000)
            else:
                dataset_size = int(size.replace('k', '000'))
            
            # Append a dictionary with the data to the list
            data_list.append({
                'seed': seed,
                'lr': lr,
                'dataset_size': dataset_size,
                'post_attack_asr': score
            })

# Create the DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Save the DataFrame to a CSV file
# df.to_csv("results/summarized_results/llama-2-ft_repnoise_0.001_beta_chattemp_corrloss_disjoint_attack_test_results.csv", index=False)
# df.to_csv(f"results/summarized_results/llama-2-chat-hf_chattemp_corrloss_disjoint_attack_test_results.csv", index=False)
# df.to_csv(f"results/summarized_results/llama-2-ft_repnoise_0.001_beta_extra1.csv", index=False)
# df.to_csv(f"results/summarized_results/llama-2-ft_repnoise_0.001_beta_{lr}_{size}.csv", index=False)
df.to_csv(f"results/summarized_results_deduplicate/repnoise_corrloss_disjoint_attack_test_deduplicate.csv", index=False)


print("Data has been successfully saved to 'combined_results.csv'")

Data has been successfully saved to 'combined_results.csv'
